# Part 1: Scraping data from Wikipidea

### Import Libraries

In [7]:
import requests # library to handle requests
import bs4 as bs
import urllib.request

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

### Extract postal codes

In [19]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"  # Wiki url
res = requests.get(url)
soup = bs.BeautifulSoup(res.content,'html.parser')
table = soup.find_all('table')[0] 
columns = ['Postcode', 'Borough', 'Neighbourhood']
temp_df = pd.read_html(str(table))[0]

temp_df.columns = temp_df.iloc[0]
temp_df = temp_df.reindex(temp_df.index.drop(0)).reset_index(drop=True)
print(temp_df.shape)
temp_df.head()    # check temp data

(288, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


 ### Process the temporary data

In [18]:
df=temp_df            
df = df[df.Borough != 'Not assigned']  
# intergrate postal codes with more than one neighbour 
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
print(df.shape)
df.head()         # check processed data

(103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Save Toronto postcode data

In [4]:
df.to_csv('Toronto Postcode.csv',index=False)